In [3]:
# Must load the xlsx file containing captures.
# The columns 'Spectro_Start' and 'Spectro_end' along with 'Date' denote what .sig files to collect SpectraVistaData from.
# Output csv will contain columns: Date, Variety, Vines_ID, Spectro, Row, WV[0,..,N]
import pandas as pd
import sys
sys.path.append("../..")
from SpectralUtilities import SpectraVistaData
import datetime
from os.path import isdir, join, isfile
from os import listdir
from collections import defaultdict
## Utility methods to accomplish goal
# Changes the XLSX datetime format into mmddyy, no spaces
format_date = lambda i : i.strftime('%m%d%y')
# Changes the data sample to map into the .sig filenames
format_spectro = lambda i : f"{str(int(i)):>04}"
# Drop rows where Spectro_Start and Spectr_End is not a valid number
spec_start = "Spectro_Start"
spec_end = "Spectr_End"
is_not_num = lambda i : not isinstance(i, (int,float))

# The .sig file name format is as follows: "HR.<DATE>.<ID>.sig"
# Note The .sig files are found in Spectro_files_<DATASET>/<DATE> in some cases instead of DATE -> IOP4
gen_sig_name = lambda date, capture_id : f"HR.{date}.{capture_id}.sig"

# Must investigate why this is necessary for some varieties
def remove_outlier(dict_list:defaultdict) -> defaultdict:
    first_key_length = len(dict_list[next(iter(dict_list))])
    filtered_dict = {key: value for key, value in dict_list.items() if len(value) == first_key_length}
    return filtered_dict

In [ ]:
parent_dir = "/home/ben/Documents/repos/Spectro_projects"
dataset_names = ["/home/ben/Documents/repos/Spectro_projects/1_Metabo/Hypervid_Spectro_Metabo_Record.xlsx",
                 "/home/ben/Documents/repos/Spectro_projects/2_OVB/Hypervid_Spectro_OVB_Record.xlsx",
                 "/home/ben/Documents/repos/Spectro_projects/3_Cab/Hypervid_Spectro_Cab_Record.xlsx"]
collections = ["1_Metabo","2_OVB","3_Cab"]
# [Date, Variety, Vines_ID, Spectro, Row, WV...]
# wavelengths... [337.2, 338.7, 340.2, 341.6, 343.1, 344.5, 346.0, 347.5, 348.9, 350.4, 351.9, 353.3, 354.8, 356.3, 357.7, 359.2, 360.7, 362.1, 363.6, 365.0, 366.5, 368.0, 369.5, 370.9, 372.4, 373.8, 375.3, 376.8, 378.2, 379.7, 381.2, 382.6, 384.1, 385.6, 387.0, 388.5, 390.0, 391.4, 392.9, 394.3, 395.8, 397.3, 398.7, 400.2, 401.6, 403.1, 404.6, 406.0, 407.5, 409.0, 410.4, 411.9, 413.3, 414.8, 416.2, 417.7, 419.1, 420.6, 422.1, 423.5, 425.0, 426.4, 427.9, 429.3, 430.8, 432.2, 433.7, 435.1, 436.6, 438.0, 439.5, 441.0, 442.4, 443.9, 445.3, 446.8, 448.2, 449.6, 451.1, 452.5, 454.0, 455.4, 456.9, 458.3, 459.8, 461.2, 462.6, 464.1, 465.5, 467.0, 468.4, 469.9, 471.3, 472.7, 474.2, 475.6, 477.0, 478.5, 479.9, 481.3, 482.8, 484.2, 485.6, 487.1, 488.5, 489.9, 491.4, 492.8, 494.2, 495.6, 497.1, 498.5, 499.9, 501.4, 502.8, 504.2, 505.6, 507.1, 508.5, 509.9, 511.3, 512.8, 514.2, 515.6, 517.0, 518.4, 519.8, 521.3, 522.7, 524.1, 525.5, 526.9, 528.3, 529.8, 531.2, 532.6, 534.0, 535.4, 536.8, 538.2, 539.6, 541.0, 542.4, 543.8, 545.2, 546.7, 548.1, 549.5, 550.9, 552.3, 553.7, 555.1, 556.5, 557.9, 559.3, 560.7, 562.1, 563.5, 564.9, 566.3, 567.7, 569.1, 570.5, 571.8, 573.2, 574.6, 576.0, 577.4, 578.8, 580.2, 581.6, 583.0, 584.4, 585.8, 587.1, 588.5, 589.9, 591.3, 592.7, 594.1, 595.4, 596.8, 598.2, 599.6, 601.0, 602.3, 603.7, 605.1, 606.5, 607.9, 609.2, 610.6, 612.0, 613.4, 614.7, 616.1, 617.5, 618.9, 620.2, 621.6, 623.0, 624.3, 625.7, 627.1, 628.5, 629.8, 631.2, 632.5, 633.9, 635.3, 636.6, 638.0, 639.4, 640.7, 642.1, 643.5, 644.8, 646.2, 647.5, 648.9, 650.2, 651.6, 653.0, 654.3, 655.7, 657.0, 658.4, 659.7, 661.1, 662.4, 663.8, 665.2, 666.5, 667.8, 669.2, 670.5, 671.9, 673.2, 674.6, 675.9, 677.3, 678.6, 680.0, 681.3, 682.7, 684.0, 685.3, 686.7, 688.0, 689.3, 690.7, 692.0, 693.4, 694.7, 696.0, 697.4, 698.7, 700.0, 701.4, 702.7, 704.0, 705.4, 706.7, 708.0, 709.4, 710.7, 712.0, 713.3, 714.7, 716.0, 717.3, 718.6, 720.0, 721.3, 722.6, 723.9, 725.2, 726.6, 727.9, 729.2, 730.5, 731.8, 733.2, 734.5, 735.8, 737.1, 738.4, 739.7, 741.0, 742.4, 743.7, 745.0, 746.3, 747.6, 748.9, 750.2, 751.5, 752.8, 754.1, 755.4, 756.7, 758.0, 759.3, 760.6, 761.9, 763.2, 764.5, 765.8, 767.1, 768.4, 769.7, 771.0, 772.3, 773.6, 774.9, 776.2, 777.5, 778.8, 780.1, 781.3, 782.6, 783.9, 785.2, 786.5, 787.8, 789.1, 790.3, 791.6, 792.9, 794.2, 795.5, 796.7, 798.0, 799.3, 800.6, 801.8, 803.1, 804.4, 805.7, 806.9, 808.2, 809.5, 810.7, 812.0, 813.3, 814.5, 815.8, 817.1, 818.3, 819.6, 820.9, 822.1, 823.4, 824.6, 825.9, 827.2, 828.4, 829.7, 830.9, 832.2, 833.4, 834.7, 835.9, 837.2, 838.4, 839.7, 840.9, 842.2, 843.4, 844.7, 845.9, 847.1, 848.4, 849.6, 850.9, 852.1, 853.3, 854.6, 855.8, 857.0, 858.3, 859.5, 860.7, 862.0, 863.2, 864.4, 865.7, 866.9, 868.1, 869.3, 870.6, 871.8, 873.0, 874.2, 875.4, 876.7, 877.9, 879.1, 880.3, 881.5, 882.7, 884.0, 885.2, 886.4, 887.6, 888.8, 890.0, 891.2, 892.4, 893.6, 894.8, 896.0, 897.2, 898.4, 899.6, 900.8, 902.0, 903.2, 904.4, 905.6, 906.8, 908.0, 909.2, 910.4, 911.6, 912.8, 914.0, 915.2, 916.4, 917.5, 918.7, 919.9, 921.1, 922.3, 923.5, 924.7, 925.8, 927.0, 928.2, 929.4, 930.5, 931.7, 932.9, 934.1, 935.2, 936.4, 937.6, 938.8, 939.9, 941.1, 942.3, 943.4, 944.6, 945.8, 946.9, 948.1, 949.3, 950.4, 951.6, 952.8, 953.9, 955.1, 956.2, 957.4, 958.5, 959.7, 960.9, 962.0, 963.2, 964.3, 965.5, 966.6, 967.8, 969.0, 970.1, 971.2, 972.4, 973.6, 974.7, 975.9, 977.0, 978.2, 979.3, 980.5, 981.6, 982.8, 983.9, 985.0, 986.2, 987.3, 988.5, 989.6, 990.8, 991.9, 993.0, 994.2, 995.3, 996.5, 997.6, 998.8, 999.9, 1001.1, 1002.2, 1003.3, 1004.5, 1005.6, 1006.8, 1007.9, 1009.0, 1010.2, 1011.3, 1012.5, 969.3, 973.2, 977.0, 980.8, 984.6, 988.5, 992.3, 996.1, 999.9, 1003.8, 1007.6, 1011.4, 1015.2, 1019.0, 1022.9, 1026.7, 1030.5, 1034.3, 1038.1, 1041.9, 1045.7, 1049.6, 1053.4, 1057.2, 1061.0, 1064.8, 1068.6, 1072.4, 1076.2, 1080.0, 1083.8, 1087.6, 1091.4, 1095.2, 1099.0, 1102.8, 1106.6, 1110.4, 1114.2, 1118.0, 1121.8, 1125.5, 1129.3, 1133.1, 1136.9, 1140.7, 1144.5, 1148.2, 1152.0, 1155.8, 1159.6, 1163.4, 1167.1, 1170.9, 1174.7, 1178.5, 1182.2, 1186.0, 1189.8, 1193.5, 1197.3, 1201.1, 1204.8, 1208.6, 1212.4, 1216.1, 1219.9, 1223.6, 1227.4, 1231.2, 1234.9, 1238.7, 1242.4, 1246.2, 1249.9, 1253.7, 1257.4, 1261.2, 1264.9, 1268.7, 1272.4, 1276.1, 1279.9, 1283.6, 1287.4, 1291.1, 1294.8, 1298.6, 1302.3, 1306.0, 1309.8, 1313.5, 1317.2, 1321.0, 1324.7, 1328.4, 1332.1, 1335.9, 1339.6, 1343.3, 1347.0, 1350.7, 1354.4, 1358.2, 1361.9, 1365.6, 1369.3, 1373.0, 1376.7, 1380.4, 1384.1, 1387.8, 1391.6, 1395.2, 1398.9, 1402.7, 1406.3, 1410.1, 1413.8, 1417.4, 1421.1, 1424.8, 1428.5, 1432.2, 1435.9, 1439.6, 1443.3, 1447.0, 1450.6, 1454.3, 1458.0, 1461.7, 1465.4, 1469.0, 1472.7, 1476.4, 1480.1, 1483.7, 1487.4, 1491.1, 1494.7, 1498.4, 1502.1, 1505.7, 1509.4, 1513.1, 1516.7, 1520.4, 1524.0, 1527.7, 1531.3, 1535.0, 1538.7, 1542.3, 1545.9, 1549.6, 1553.2, 1556.9, 1560.5, 1564.2, 1567.8, 1571.4, 1575.1, 1578.7, 1582.4, 1586.0, 1589.6, 1593.3, 1596.9, 1600.5, 1604.2, 1607.8, 1611.4, 1615.0, 1618.6, 1622.3, 1625.9, 1629.5, 1633.1, 1636.7, 1640.3, 1643.9, 1647.6, 1651.2, 1654.8, 1658.4, 1662.0, 1665.6, 1669.2, 1672.8, 1676.4, 1680.0, 1683.6, 1687.2, 1690.8, 1694.4, 1697.9, 1701.5, 1705.1, 1708.7, 1712.3, 1715.9, 1719.5, 1723.0, 1726.6, 1730.2, 1733.8, 1737.3, 1740.9, 1744.5, 1748.1, 1751.6, 1755.2, 1758.8, 1762.3, 1765.9, 1769.4, 1773.0, 1776.6, 1780.1, 1783.7, 1787.2, 1790.8, 1794.3, 1797.9, 1801.4, 1805.0, 1808.5, 1812.1, 1815.6, 1819.1, 1822.7, 1826.2, 1829.7, 1833.3, 1836.8, 1840.3, 1843.9, 1847.4, 1850.9, 1854.4, 1858.0, 1861.5, 1865.0, 1868.5, 1872.0, 1875.6, 1879.1, 1882.6, 1886.1, 1889.6, 1893.1, 1896.6, 1900.1, 1903.6, 1907.1, 1899.0, 1901.8, 1904.7, 1907.5, 1910.3, 1913.1, 1915.9, 1918.7, 1921.5, 1924.2, 1927.0, 1929.8, 1932.6, 1935.4, 1938.2, 1940.9, 1943.7, 1946.5, 1949.2, 1952.0, 1954.7, 1957.5, 1960.2, 1963.0, 1965.7, 1968.5, 1971.2, 1973.9, 1976.7, 1979.4, 1982.1, 1984.8, 1987.6, 1990.3, 1993.0, 1995.7, 1998.4, 2001.1, 2003.8, 2006.5, 2009.2, 2011.9, 2014.6, 2017.3, 2020.0, 2022.6, 2025.3, 2028.0, 2030.7, 2033.3, 2036.0, 2038.7, 2041.3, 2044.0, 2046.6, 2049.3, 2051.9, 2054.6, 2057.2, 2059.8, 2062.5, 2065.1, 2067.7, 2070.4, 2073.0, 2075.6, 2078.2, 2080.8, 2083.4, 2086.0, 2088.6, 2091.2, 2093.8, 2096.4, 2099.0, 2101.6, 2104.2, 2106.8, 2109.4, 2111.9, 2114.5, 2117.1, 2119.6, 2122.2, 2124.8, 2127.3, 2129.9, 2132.4, 2135.0, 2137.5, 2140.1, 2142.6, 2145.1, 2147.7, 2150.2, 2152.7, 2155.2, 2157.8, 2160.3, 2162.8, 2165.3, 2167.8, 2170.3, 2172.8, 2175.3, 2177.8, 2180.3, 2182.8, 2185.3, 2187.8, 2190.3, 2192.8, 2195.2, 2197.7, 2200.2, 2202.6, 2205.1, 2207.6, 2210.0, 2212.5, 2214.9, 2217.4, 2219.8, 2222.2, 2224.7, 2227.1, 2229.6, 2232.0, 2234.4, 2236.8, 2239.2, 2241.7, 2244.1, 2246.5, 2248.9, 2251.3, 2253.7, 2256.1, 2258.5, 2260.9, 2263.3, 2265.7, 2268.1, 2270.4, 2272.8, 2275.2, 2277.6, 2279.9, 2282.3, 2284.6, 2287.0, 2289.4, 2291.7, 2294.1, 2296.4, 2298.8, 2301.1, 2303.4, 2305.8, 2308.1, 2310.4, 2312.8, 2315.1, 2317.4, 2319.7, 2322.0, 2324.3, 2326.6, 2328.9, 2331.2, 2333.5, 2335.8, 2338.1, 2340.4, 2342.7, 2345.0, 2347.3, 2349.5, 2351.8, 2354.1, 2356.4, 2358.6, 2360.9, 2363.1, 2365.4, 2367.6, 2369.9, 2372.1, 2374.4, 2376.6, 2378.9, 2381.1, 2383.3, 2385.5, 2387.8, 2390.0, 2392.2, 2394.4, 2396.6, 2398.8, 2401.1, 2403.2, 2405.4, 2407.6, 2409.8, 2412.0, 2414.2, 2416.4, 2418.6, 2420.8, 2422.9, 2425.1, 2427.3, 2429.4, 2431.6, 2433.8, 2435.9, 2438.1, 2440.2, 2442.4, 2444.5, 2446.7, 2448.8, 2450.9, 2453.1, 2455.2, 2457.3, 2459.5, 2461.6, 2463.7, 2465.8, 2467.9, 2470.0, 2472.1, 2474.2, 2476.3, 2478.4, 2480.5, 2482.6, 2484.7, 2486.8, 2488.9, 2490.9, 2493.0, 2495.1, 2497.2, 2499.2, 2501.3, 2503.4, 2505.4, 2507.5, 2509.5, 2511.6, 2513.6, 2515.6, 2517.7]

out_df = defaultdict(list)

for idx, dataset_name in enumerate(dataset_names):
    try:
        xl = pd.read_excel(dataset_name)
        # Clean the DF
        xl.drop(xl[xl[spec_start].map(is_not_num)].index,inplace=True)
        xl.drop(xl[xl[spec_end].map(is_not_num)].index,inplace=True)
        xl["Date"] = xl["Date"].map(format_date)
        # Loop through rows and create a list of filenames based on spec capture ranges
        for i in xl.index:
            capture_date = xl.loc[i,"Date"]
            capture_ids = [int(xl.loc[i,spec_start]) + x for x in range((int(xl.loc[i,spec_end]) - int(xl.loc[i,spec_start])))]
            for id in capture_ids:
                sig_file_name = gen_sig_name(date=capture_date,capture_id=format_spectro(id))
                sig_file_path = join(f"{parent_dir}",collections[idx],f"Spectro_files_{collections[idx][2:]}",capture_date,sig_file_name)
                # print(sig_file_name)
                print(sig_file_path)
                if not isfile(sig_file_path):
                    # Here we will iterate through IOP sub folders
                    # sig_file_path = join(parent_dir,f"Spectro_files_{dataset_name}",capture_date,sig_file_name)
                    # if not isfile(sig_file_path):
                    print(f"skipping {sig_file_path}")
                    continue
                # Read in the sig file and accumulate the reflectance data
                spectra_data = SpectraVistaData(file_path)
                spectra_data.data.set_index("wv", inplace=True)

                out_df['Date'].append(capture_date)
                out_df['Variety'].append(xl.loc[i,'Field'])
                out_df['Vines_ID'].append(xl.loc[i,'Vines_ID'])
                out_df['Spectro'].append(id)
                out_df['Row'].append(xl.loc[i,'Row number'])
                
                for wv in spectra_data.data.index.to_list():
                    refl = spectra_data.data.loc[wv,'refl']
                    out_df[wv].append(refl)
        df = pd.DataFrame.from_dict(remove_outlier(out_df))
        df.to_csv(f"{collections[idx]}_reflectance.csv")
        print(f"Saved")
    except Exception as e:
        print(f"{e}")                    